# Chapter 15 - Object Mappers



## The Object Mapper pattern

- Copy the value of an objects properties into the properties of another object
    * i.e. Model to DTO

In [5]:
classDiagram
    class IMapper~TSource, TDestination~ {
        +map(TSource)  TDestination
    }

### Project - Mapper

Simple example. See `C15\Mapper` for full example.

In [6]:
public interface IMapper<TSource, TDestination>
{
    TDestination Map(TSource entity);
}

//DTO
public record class ProductDTO(int Id, string Name, int QuantityInStock);


public class ProductMapper : IMapper<Product, ProductDetails>
{
    public ProductDetails Map(Product entity)
        => new(entity.Id ?? default, entity.Name, entity.QuantityInStock);
}

public class Product
{
    public Product(string name, int quantityInStock, int shelfNumber, int? id = null)
    {
        Name = name ?? throw new ArgumentNullException(nameof(name));
        QuantityInStock = quantityInStock;
        Id = id;
        ShelfNumber = shelfNumber;
    }

    public int? Id { get; init; }
    public string Name { get; init; }
    public string Description { get; init; }
    public int ShelfNumber { get; init; }
    public int QuantityInStock { get; private set; }

    public void AddStock(int amount)
    {
        if (amount == 0) { return; }
        QuantityInStock += amount;
    }

    public void RemoveStock(int amount)
    {
        if (amount == 0) { return; }
        QuantityInStock -= amount;
    }
}

var mapper = new ProductMapper();

var products = new List<Product>() 
{
    new Product(
        id: 1,
        name: "Banana",
        shelfNumber: 1,
        quantityInStock: 50
    ),
    new Product(
        id: 2,
        name: "Apple",
        shelfNumber: 1,
        quantityInStock: 20
    ),
    new Product(
        id: 3,
        name: "Habanero Pepper",
        shelfNumber: 2,
        quantityInStock: 10
    )
};

display( System.Text.Json.JsonSerializer.Serialize(products) );

var data = products.Select( p => mapper.Map(p) );

display( System.Text.Json.JsonSerializer.Serialize(data) );


[{"Id":1,"Name":"Banana","Description":null,"ShelfNumber":1,"QuantityInStock":50},{"Id":2,"Name":"Apple","Description":null,"ShelfNumber":1,"QuantityInStock":20},{"Id":3,"Name":"Habanero Pepper","Description":null,"ShelfNumber":2,"QuantityInStock":10}]

[{"Id":1,"Name":"Banana","QuantityInStock":50},{"Id":2,"Name":"Apple","QuantityInStock":20},{"Id":3,"Name":"Habanero Pepper","QuantityInStock":10}]

## Too many dependencies

- This pattern can quickly become messay with a lot of dependencies
    * REduced readability
    * Decreased maintainability
    * Mock heavy tests
    * Inflated scope

- Rule of thumb: 3 or less dependencies

### Aggregate Services Pattern

- Can be used to reduce the number of dependencies
- Putting related services together into 1

### Facade

- Can also use the facade pattern for this as well

## Dynamic MappingService

In [ ]:
public interface IMappingService
{
    TDestination Map<TSource, TDestination>(TSource entity);
}

public class MapperNotFoundException : Exception
{
    public MapperNotFoundException(Type source, Type destination)
        : base($"No Mapper from '{source}' to '{destination}' was found.")
    {
    }
}

public class ServiceLocatorMappingService : IMappingService
{
    private readonly IServiceProvider _serviceProvider;
    public ServiceLocatorMappingService(IServiceProvider serviceProvider)
    {
        _serviceProvider = serviceProvider ?? throw new ArgumentNullException(nameof(serviceProvider));
    }

    public TDestination Map<TSource, TDestination>(TSource entity)
    {
        var mapper = _serviceProvider.GetService<IMapper<TSource, TDestination>>();
        if (mapper == null)
        {
            throw new MapperNotFoundException(typeof(TSource), typeof(TDestination));
        }
        return mapper.Map(entity);
    }
}

## AutoMapper

- OpenSource tool that does the mapping for us
- Maps using conventions
 
```
dotnet add package AutoMapper
```


In [1]:
#r "nuget:AutoMapper"

Installed Packages AutoMapper, 14.0.0

### Create a profile

```csharp
using AutoMapper;
public class WebProfile : Profile
{
    public WebProfile()
    {
        CreateMap<Product, ProductDetails>();
        CreateMap<NotEnoughStockException, NotEnoughStock>();
        CreateMap<ProductNotFoundException, ProductNotFound>();
    }
}
```

### Injection extension

```csharp
builder.Services.AddAutoMapper(typeof(WebProfile).Assembly);
```

### Usage example

```csharp
app.MapGet("/products", async (
    IProductRepository productRepository, 
    IMapper mapper, 
    CancellationToken cancellationToken) =>
{
    var products = await productRepository.AllAsync(cancellationToken);
    return products.Select(p => mapper.Map<Product, ProductDetails>(p));
});
```

Use to map from EF core 

```csharp
public IEnumerable<ProductDto> GetAllProducts()
{
    return _mapper.ProjectTo<ProductDto>(_db.Products);
}
```

## Mapperly

- Newer object mapper library that leverages source generation to make it fast
    * Code is generated at compile time

```
dotnet package add  Riok.Mapperly 
```

In [1]:
#r "nuget:Riok.Mapperly"

Installed Packages Riok.Mapperly, 4.2.1

Editing this property in the project file allows you to view the generated source.

```
<PropertyGroup>
 <EmitCompilerGeneratedFiles>true</EmitCompilerGeneratedFiles>
</PropertyGroup>
```

See [C15\Mapperly\Web\Program.cs](C15\Mapperly\Web\Program.cs) for full example using this.